In [1]:
!pip install fairseq

     |████████████████████████████████| 307kB 8.2MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2015628 sha256=f3f5b3219b1115a6a0ec6dde5f023c5002eee9b0aca59eed0f6de800a73a16ac
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


In [0]:
%%bash
mkdir en_de_script
mkdir de_en_script

Кладем скрипты предпобработки и вашу модель по нужным папкам.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/Bidir translation/prepare-iwslt17-my_de_en.sh'\
    '/content/de_en_script/prepare-iwslt17-my_de_en.sh'
!cp '/content/gdrive/My Drive/Bidir translation/prepare-iwslt17-my_en_de.sh'\
    '/content/en_de_script/prepare-iwslt17-my_en_de.sh'
# Ваша модель
!cp '/content/gdrive/My Drive/Bidir translation/simple_lstm.py' \
    '/usr/local/lib/python3.6/dist-packages/fairseq/models/simple_lstm.py'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Предобработка датасета для de-en

In [6]:
%%bash
cd de_en_script/
bash prepare-iwslt17-my_de_en.sh

Cloning Moses github repository (for tokenization scripts)...
Cloning Subword NMT repository (for BPE pre-processing)...
Data successfully downloaded.
de-en/
de-en/IWSLT17.TED.dev2010.de-en.de.xml
de-en/IWSLT17.TED.dev2010.de-en.en.xml
de-en/IWSLT17.TED.tst2010.de-en.de.xml
de-en/IWSLT17.TED.tst2010.de-en.en.xml
de-en/IWSLT17.TED.tst2011.de-en.de.xml
de-en/IWSLT17.TED.tst2011.de-en.en.xml
de-en/IWSLT17.TED.tst2012.de-en.de.xml
de-en/IWSLT17.TED.tst2012.de-en.en.xml
de-en/IWSLT17.TED.tst2013.de-en.de.xml
de-en/IWSLT17.TED.tst2013.de-en.en.xml
de-en/IWSLT17.TED.tst2014.de-en.de.xml
de-en/IWSLT17.TED.tst2014.de-en.en.xml
de-en/IWSLT17.TED.tst2015.de-en.de.xml
de-en/IWSLT17.TED.tst2015.de-en.en.xml
de-en/README
de-en/train.tags.de-en.de
de-en/train.tags.de-en.en
pre-processing train data...


pre-processing valid/test data...
orig/de-en/IWSLT17.TED.dev2010.de-en.de.xml iwslt17.tokenized.de-en/tmp/IWSLT17.TED.dev2010.de-en.de

orig/de-en/IWSLT17.TED.tst2010.de-en.de.xml iwslt17.tokenized.de

Cloning into 'mosesdecoder'...
Cloning into 'subword-nmt'...
--2020-03-05 12:21:31--  https://wit3.fbk.eu/archive/2017-01-trnted/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 217.77.80.8
Connecting to wit3.fbk.eu (wit3.fbk.eu)|217.77.80.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16751373 (16M) [application/x-gzip]
Saving to: ‘de-en.tgz’

     0K .......... .......... .......... .......... ..........  0%  792K 21s
    50K .......... .......... .......... .......... ..........  0% 1.18M 17s
   100K .......... .......... .......... .......... ..........  0% 2.35M 14s
   150K .......... .......... .......... .......... ..........  1% 2.36M 12s
   200K .......... .......... .......... .......... ..........  1% 2.35M 11s
   250K .......... .......... .......... .......... ..........  1% 2.39M 10s
   300K .......... .......... .......... .......... ..........  2% 2.41M 9s
   350K .......... .......... .......... .......... ..........  2% 49.1M 8

In [27]:
%env TEXT=de_en_script/iwslt17.tokenized.de-en
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt17.tokenized.de-en

env: TEXT=de_en_script/iwslt17.tokenized.de-en
Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/iwslt17.tokenized.de-en', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='de', srcdict=None, target_lang='en', task='translation', tensorboard_logdir='', testpref='de_en_script/iwslt17.tokenized.de-en/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='de_en_script/iwslt17.tokenized.de-en/train', user_dir=None, validpref='de_en_script/iwslt17.tokenized.de-en/valid', workers=1)
| [de] Dictionary: 8975 types
| [de] de_en_script/iwslt17.tokenized.de

Предобработка датасета для en-de

In [28]:
%%bash
cd en_de_script/
bash prepare-iwslt17-my_en_de.sh

Cloning Moses github repository (for tokenization scripts)...
Cloning Subword NMT repository (for BPE pre-processing)...
Data successfully downloaded.
en-de/
en-de/IWSLT17.TED.dev2010.en-de.de.xml
en-de/IWSLT17.TED.dev2010.en-de.en.xml
en-de/IWSLT17.TED.tst2010.en-de.de.xml
en-de/IWSLT17.TED.tst2010.en-de.en.xml
en-de/IWSLT17.TED.tst2011.en-de.de.xml
en-de/IWSLT17.TED.tst2011.en-de.en.xml
en-de/IWSLT17.TED.tst2012.en-de.de.xml
en-de/IWSLT17.TED.tst2012.en-de.en.xml
en-de/IWSLT17.TED.tst2013.en-de.de.xml
en-de/IWSLT17.TED.tst2013.en-de.en.xml
en-de/IWSLT17.TED.tst2014.en-de.de.xml
en-de/IWSLT17.TED.tst2014.en-de.en.xml
en-de/IWSLT17.TED.tst2015.en-de.de.xml
en-de/IWSLT17.TED.tst2015.en-de.en.xml
en-de/README
en-de/train.tags.en-de.de
en-de/train.tags.en-de.en
pre-processing train data...


pre-processing valid/test data...
orig/en-de/IWSLT17.TED.dev2010.en-de.en.xml iwslt17.tokenized.en-de/tmp/IWSLT17.TED.dev2010.en-de.en

orig/en-de/IWSLT17.TED.tst2010.en-de.en.xml iwslt17.tokenized.en

Cloning into 'mosesdecoder'...
Cloning into 'subword-nmt'...
--2020-03-05 12:48:38--  https://wit3.fbk.eu/archive/2017-01-trnted/texts/en/de/en-de.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 217.77.80.8
Connecting to wit3.fbk.eu (wit3.fbk.eu)|217.77.80.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16751364 (16M) [application/x-gzip]
Saving to: ‘en-de.tgz’

     0K .......... .......... .......... .......... ..........  0%  793K 21s
    50K .......... .......... .......... .......... ..........  0% 2.32M 14s
   100K .......... .......... .......... .......... ..........  0% 2.35M 11s
   150K .......... .......... .......... .......... ..........  1% 2.38M 10s
   200K .......... .......... .......... .......... ..........  1%  931K 12s
   250K .......... .......... .......... .......... ..........  1% 2.39M 11s
   300K .......... .......... .......... .......... ..........  2% 2.38M 10s
   350K .......... .......... .......... .......... ..........  2% 89.2M 

In [29]:
%env TEXT=en_de_script/iwslt17.tokenized.en-de
!fairseq-preprocess --source-lang en --target-lang de \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt17.tokenized.en-de

env: TEXT=en_de_script/iwslt17.tokenized.en-de
Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/iwslt17.tokenized.en-de', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='en', srcdict=None, target_lang='de', task='translation', tensorboard_logdir='', testpref='en_de_script/iwslt17.tokenized.en-de/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='en_de_script/iwslt17.tokenized.en-de/train', user_dir=None, validpref='en_de_script/iwslt17.tokenized.en-de/valid', workers=1)
| [en] Dictionary: 6759 types
| [en] en_de_script/iwslt17.tokenized.en

Так запускается обучение модели.

In [37]:
!cp -r data-bin \
    /content/gdrive/My\ Drive/Bidir\ translation/

cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/mosesdecoder/scripts/share/nonbreaking_prefixes/nonbreaking_prefix.et': Operation not supported
cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/mosesdecoder/scripts/training/clean-corpus-n-ratio.perl': Operation not supported
cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/subword-nmt/apply_bpe.py': Operation not supported
cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/subword-nmt/get_vocab.py': Operation not supported
cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/subword-nmt/learn_bpe.py': Operation not supported
cp: cannot create symbolic link '/content/gdrive/My Drive/Bidir translation/en_de_script/subword-nmt/learn_joint_bpe_and_vocab.py': Operation not supported


In [32]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train data-bin/iwslt17.tokenized.de-en \
    --lr 0.25 --clip-norm 0.1 --max-tokens 4000 \
    --arch tutorial_simple_lstm \
    --save-dir /content/gdrive/My\ Drive/Bidir\ translation/checkpoints/fconv

Namespace(arch='tutorial_simple_lstm', best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.1, cpu=False, criterion='cross_entropy', curriculum=0, data='data-bin/iwslt17.tokenized.de-en', dataset_impl=None, ddp_backend='c10d', decoder_dropout=0.1, decoder_embed_dim=256, decoder_hidden_dim=256, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, empty_cache_freq=0, encoder_dropout=0.1, encoder_embed_dim=256, encoder_hidden_dim=256, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, keep_interval_updates=-1, keep_last_epochs=-1, lazy_load=False, left_pad_source='True', left_pad_target='False', load_alignments=False, log_format=None, log_interval=1000, lr=[0.25], lr_scheduler='f